In [1]:
import sys
import math

from src.raytracer.raytracer import point3, vec3, color, unit
from src.raytracer.raytracer import unit, dot
from src.raytracer.raytracer import random_in_hemisphere, random_unit_vector
from src.raytracer.ray import ray
from src.raytracer import hittables, materials, textures
from src.raytracer import camera, writeimg, rweekend
from datetime import datetime
import pytz

In [2]:
def ray_color(r, world, depth):
    rec = hittables.hit_record()
    if depth <= 0:
        return color(0, 0, 0)
    
    hit_out = world.hit(r, 0.001, rweekend.infinity, rec)
    if hit_out[0]:
        rec = hit_out[1]
        
        scattered = ray()
        attenuation = vec3()
        out = rec.material.scatter(r, rec, attenuation, scattered)
        if out[0]:
            scattered = out[1]
            attenuation = out[2]
            # print('attenuation: ', str(attenuation))
            # print('scattered: ', str(scattered))
            return (attenuation * ray_color(scattered, world, depth-1))
        return color(0, 0, 0)
            
        # target = rec.p + rec.normal + random_unit_vector()  # diffuse v1
        # target = rec.p + random_in_hemisphere(rec.normal)  # diffuse v2
        # out = 0.5 * ray_color(ray(rec.p, target - rec.p), world, depth-1)
        # return out
    
    # make background gradient
    unit_direction = unit(r.direction)
    t = 0.5 * (unit_direction.y + 1.0)
    return (1.0 - t) * color(1.0, 1.0, 1.0) \
            + t * color(0.5, 0.7, 1.0)

In [3]:
def random_scene():
    world = hittables.hittable_list()
    
    ground_material = materials.lambertian(color(0.5, 0.5, 0.5))
    world.add(hittables.sphere(point3(0.0, -1000, 0.0), 1000.0, ground_material))
    
    for a in range(-2, 2):
        for b in range(-2, 2):
            choose_mat = rweekend.random_double()
            center = point3(a + 0.9 * rweekend.random_double(), 0.2, b + 0.9 * rweekend.random_double())
            
            if ((center - point3(4.0, 0.2, 0.0)).len > 0.9):
                if (choose_mat < 0.8):
                    # diffuse sphere
                    albedo = color.random() * color.random()
                    sphere_material = materials.lambertian(albedo)
                    center2 = center + vec3(0.0, rweekend.random_double(0.0, 0.5), 0.0) # add moving sphere
                    world.add(hittables.moving_sphere(center, center2, 0.0, 1.0, 0.2, sphere_material))
                elif (choose_mat < 0.95):
                    # metal sphere
                    albedo = color.random(0.5, 1.0)
                    fuzz = rweekend.random_double(0.0, 0.5)
                    sphere_material = materials.metal(albedo, fuzz)
                    world.add(hittables.sphere(center, 0.2, sphere_material))
                else:
                    # glass sphere
                    sphere_material = materials.dielectric(1.5)
                    world.add(hittables.sphere(center, 0.2, sphere_material))

    material1 = materials.dielectric(1.5)
    world.add(hittables.sphere(point3(0.0, 1.0, 0.0), 1.0, material1))
    
    material2 = materials.lambertian(color(0.4, 0.2, 0.1))
    world.add(hittables.sphere(point3(-4.0, 1.0, 0.0), 1.0, material2))
    
    material3 = materials.metal(color(0.7, 0.6, 0.5), 0.0)
    world.add(hittables.sphere(point3(4.0, 1.0, 0.0), 1.0, material3))
    
    return world
                    

In [4]:
##

In [5]:
# image params
#aspect_ratio = 3.0 / 2.0
aspect_ratio = 16.0 / 9.0
image_width = 400
#image_width = 1200
image_height = math.floor(image_width / aspect_ratio)
samples_per_pixel = 2 
max_depth = 5
# good for testing: spp 2, md 5
# spp 25, md 10, 30 min
# spp 10, md 20, 22 min
# spp 5, md 10, 6 min
# spp 2, md 5, 1.5 min
# better render spp 20, md 10, 14 min
# spp 100, md 50, 1hr15min

In [6]:
# world
#R = math.cos(rweekend.pi / 4)

#world = hittables.hittable_list()
#world = random_scene()

#material_ground = materials.lambertian(color(0.8, 0.8, 0.0))
#material_center = materials.lambertian(color(0.1, 0.2, 0.5))
#material_left = materials.dielectric(1.5)
#material_right = materials.metal(color(0.8, 0.6, 0.2), 1.0)

#material_left = materials.lambertian(color(0.0, 0.0, 1.0))
#material_right = materials.lambertian(color(1.0, 0.0, 0.0))

#world.add(hittables.sphere(point3(0.0, -100.5, -1.0), 100.0, material_ground))
#world.add(hittables.sphere(point3(0.0, 0.0, -1.0), 0.5, material_center))
#world.add(hittables.sphere(point3(-1.0, 0.0, -1.0), 0.5, material_left))
#world.add(hittables.sphere(point3(-1.0, 0.0, -1.0), -0.4, material_left))
#world.add(hittables.sphere(point3(1.0, 0.0, -1.0), 0.5, material_right))

#world.add(hittables.sphere(point3(-R, 0.0, -1.0), R, material_left))
#world.add(hittables.sphere(point3(R, 0.0, -1.0), R, material_right))

objects = hittables.hittable_list()
material = materials.lambertian(color(0.8, 0.8, 0.0))
objects.add(hittables.sphere(point3(0.0, 1.0, -1.0), 0.5, materials.lambertian(color(0.1, 0.8, 0.0))))
objects.add(hittables.sphere(point3(-1.0, 1.0, -1.0), 0.5, materials.lambertian(color(0.5, 0.4, 0.0))))
objects.add(hittables.sphere(point3(1.0, 1.0, -1.0), 0.5, materials.lambertian(color(0.7, 0.2, 0.0))))

ground_material = textures.checker_texture(color(0.2, 0.3, 0.1), color(0.9, 0.9, 0.9))
objects.add(hittables.sphere(point3(0.0, -1000, 0.0), 1000.0, materials.lambertian(ground_material)))

world = hittables.hittable_list()
world.add(hittables.bvh_node(hlist=objects, _time0=0.0, _time1=1.0))


In [7]:
# camera params
#lookfrom = point3(3,3,2)
lookfrom = point3(13,2,3)
#lookat = point3(0,0,-1)
lookat = point3(0,0,0)
vup = vec3(0,1,0)
#dist_to_focus = (lookfrom-lookat).len
dist_to_focus = 10.0
#aperture = 2.0
aperture = 0.1
cam = camera.camera(lookfrom, lookat, vup, 20.0, aspect_ratio, aperture, dist_to_focus, 0.0, 1.0)

In [8]:
tz_CH = pytz.timezone('America/Chicago') 
print("start time: ", datetime.now(tz_CH).strftime("%H:%M:%S"))

start time:  20:53:56


In [9]:
# render image
outimg = writeimg.writeppm(image_width, image_height,
                           'outfile.ppm', 'P3', 255)
outimg.write_head()
for j in range(image_height-1, -1, -1):
    sys.stdout.write("\r%d%%" % j)
    sys.stdout.flush()
    for i in range(0, image_width):
        pixel_color = color(0, 0, 0)
        for s in range(0, samples_per_pixel):
            u = float(i + rweekend.random_double())/(image_width - 1)
            v = float(j + rweekend.random_double())/(image_height - 1)
            r = cam.get_ray(u, v)
            pixel_color += ray_color(r, world, max_depth)
            #print("pixel_color", pixel_color)
        outimg.write_color(pixel_color, samples_per_pixel)
sys.stdout.write("done")

0%%%done

In [10]:
outimg.check_valid()

[True, 'params OK']

In [11]:
outimg.write_color_file()

In [12]:
print("end time: ", datetime.now(tz_CH).strftime("%H:%M:%S"))

end time:  20:55:14
